<h2>Preprocessing, Yay!</h2>

In [1]:
#importing dependencies
import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
#!pip install wordcloud
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [2]:
import pickle

In [3]:
df = pd.read_csv('twitter_parsed_dataset.csv')
df

,index,id,Text,Annotation,oh_label
0,5.74948705591165E+017,5.74948705591165E+017,@halalflaws @biebervalue @greenlinerzjm I read...,none,0.0
1,5.71917888690393E+017,5.71917888690393E+017,@ShreyaBafna3 Now you idiots claim that people...,none,0.0
2,3.90255841338601E+017,3.90255841338601E+017,"RT @Mooseoftorment Call me sexist, but when I ...",sexism,1.0
3,5.68208850655916E+017,5.68208850655916E+017,"@g0ssipsquirrelx Wrong, ISIS follows the examp...",racism,1.0
4,5.75596338802373E+017,5.75596338802373E+017,#mkr No No No No No No,none,0.0
...,...,...,...,...,...
16846,5.75606766236475E+017,5.75606766236475E+017,"Feeling so sorry for the girls, they should be...",none,0.0
16847,5.72333822886326E+017,5.72333822886326E+017,#MKR 'pretty good dishes we're happy with' - O...,none,0.0
16848,5.72326950057845E+017,5.72326950057845E+017,RT @colonelkickhead: Deconstructed lemon tart!...,none,0.0
16849,5.74799612642357E+017,5.74799612642357E+017,@versacezaynx @nyazpolitics @greenlinerzjm You...,none,0.0


In [4]:
df = df.dropna()

In [5]:
print(df['Text'].iloc[0], "\n")
print(df['Text'].iloc[1], "\n")
print(df['Text'].iloc[2], "\n")
print(df['Text'].iloc[3], "\n")
print(df['Text'].iloc[4], "\n")

@halalflaws @biebervalue @greenlinerzjm I read them in context.No change in meaning. The history of Islamic slavery. https://t.co/xWJzpSodGj 

@ShreyaBafna3 Now you idiots claim that people who tried to stop him from becoming a terrorist made him a terrorist. Islamically brain dead. 

RT @Mooseoftorment Call me sexist, but when I go to an auto place, I'd rather talk to a guy 

@g0ssipsquirrelx Wrong, ISIS follows the example of Mohammed and the Quran exactly. 

#mkr No No No No No No 



In [6]:
df2 = pd.read_excel('HateReviews.xlsx')
df2

,Text,oh_label
0,Not all Muslims are violent.,0
1,To be or not to be,0
2,You're so stupid!,0
3,What happens if machines take over the world?,0
4,Women have as much a right to education as men.,0
5,This is not a hate speech.,0
6,I don't hate people,0
7,Kill yourself nigger.,1
8,My Grandma's a Muslim. Such a kind woman.,0
9,Black people are literally a subspecies. You c...,1


In [9]:
ldf = pd.concat([df, df2], axis = 0, ignore_index=True)
ldf.tail(15)

,index,id,Text,Annotation,oh_label
16868,NaN,NaN,I hate how my favorite show got canceled; it w...,NaN,0.0
16869,NaN,NaN,He hates the feeling of being unprepared for a...,NaN,0.0
16870,NaN,NaN,She hates the thought of missing out on the co...,NaN,0.0
16871,NaN,NaN,I hate when my phone battery dies in the middl...,NaN,0.0
16872,NaN,NaN,"She hates getting caught in traffic, especiall...",NaN,0.0
16873,NaN,NaN,He hates that the recipe calls for so many ing...,NaN,0.0
16874,NaN,NaN,They hate how the new software update changed ...,NaN,0.0
16875,NaN,NaN,I hate the feeling of missing out on a good bo...,NaN,0.0
16876,NaN,NaN,She hates when people interrupt her while she’...,NaN,0.0
16877,NaN,NaN,"He hates the cold weather, so he prefers to st...",NaN,0.0


In [10]:
#defining a function to clean the tweets' text
def clean(text):
    text = str(text).lower()
    text = re.sub(r"https?://\S+|www\.\S+", "", text, flags = re.MULTILINE)
    test = re.sub(r"\@w+|\#", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    tweet_tokens = word_tokenize(text)
    text = [word for word in text.split(" ") if word not in stop_words]
    text = " ".join(text)
    return text

ldf.Text = ldf["Text"].apply(clean)
ldf.tail()

,index,id,Text,Annotation,oh_label
16878,NaN,NaN,hate favorite restaurant long wait times weekends,NaN,0.0
16879,NaN,NaN,hate sound nails chalkboard makes skin crawl,NaN,0.0
16880,NaN,NaN,hates dishes big meal least favorite chore,NaN,0.0
16881,NaN,NaN,even let clueless fools make decisions theyre ...,NaN,1.0
16882,NaN,NaN,frustrating deal brainless individuals cant gr...,NaN,0.0


In [11]:
lemmatizer = WordNetLemmatizer()
def lemmatizing(data):
    tweet = [lemmatizer.lemmatize(w) for w in data]
    return data

In [12]:
ldf['Text'] = ldf['Text'].apply(lambda x: lemmatizing(x))

In [13]:
ldf['Text']

0        halalflaws biebervalue greenlinerzjm read cont...
1        shreyabafna3 idiots claim people tried stop be...
2        rt mooseoftorment call sexist go auto place id...
3        g0ssipsquirrelx wrong isis follows example moh...
4                                                      mkr
                               ...                        
16878    hate favorite restaurant long wait times weekends
16879         hate sound nails chalkboard makes skin crawl
16880           hates dishes big meal least favorite chore
16881    even let clueless fools make decisions theyre ...
16882    frustrating deal brainless individuals cant gr...
Name: Text, Length: 16883, dtype: object

In [14]:
ldf['oh_label'].value_counts()

oh_label
0.0    11531
1.0     5352
Name: count, dtype: int64

<h2>Creating a Bigram language model </h2>

In [ ]:
#initialising the vectoriser

vect = TfidfVectorizer(ngram_range=(1,2)).fit(ldf['Text'])

In [ ]:
feature_names = vect.get_feature_names_out()
print("Number of features: {} \n".format(len(feature_names)))
print("First 20 features: \n{}".format(feature_names[0:20]))

Number of features: 128343 

First 20 features: 
['000' '000 mkr' '00simmerforlife' '00simmerforlife im' '01151900'
 '01151900 soon' '02' '02 feb' '05' '05 sb' '06jank'
 '06jank patrickosgood' '0cclus' '0cclus think' '0rwellian'
 '0rwellian labor' '0xabad1dea' '0xabad1dea amazon' '0xabad1dea bm'
 '0xabad1dea certain']


<h2>Trigram Language Model

In [15]:
#initialising the vectoriser

vect2 = TfidfVectorizer(ngram_range=(1,3)).fit(ldf['Text'])

In [16]:
feature_names = vect2.get_feature_names_out()
print("Number of features: {} \n".format(len(feature_names)))
print("First 20 features: \n{}".format(feature_names[0:20]))

Number of features: 239648 

First 20 features: 
['000' '000 mkr' '00simmerforlife' '00simmerforlife im'
 '00simmerforlife im sexist' '01151900' '01151900 soon' '02' '02 feb'
 '02 feb 1100' '05' '05 sb' '05 sb getting' '06jank'
 '06jank patrickosgood' '06jank patrickosgood blah'
 '06jank patrickosgood erdogan' '06jank patrickosgood evidence'
 '06jank patrickosgood give' '06jank patrickosgood schools']


<h2>Defining training data for the model</h2>

In [17]:
X = ldf['Text']
Y = ldf['oh_label']

X = vect2.transform(X)

In [18]:
X

<16883x239648 sparse matrix of type '<class 'numpy.float64'>'
	with 407618 stored elements in Compressed Sparse Row format>

In [19]:
pickle.dump(vect2, open("hatemodel/services/vectoriser.pkl", 'wb'))

<h2> Guess I'll have to oversample, too. Fun.</h2>

In [20]:
#oversampling the existing data with ADASYN
from imblearn.over_sampling import ADASYN
X_resampled, y_resampled = ADASYN().fit_resample(X, Y) # Now X_resampled and y_resampled contain the oversampled data

In [21]:
#dividing the data into training and testing data
x_train, x_test, y_train, y_test = train_test_split(X_resampled,y_resampled, test_size = 0.2, random_state = 42)

In [22]:
y_train.shape

(17778,)

In [23]:
y_test.shape

(4445,)

In [24]:
x_test

<4445x239648 sparse matrix of type '<class 'numpy.float64'>'
	with 132590 stored elements in Compressed Sparse Row format>

<h2>Building the Logistic Regression Model</h2>

In [ ]:
#Building the Logistic Regression Model
logreg = LogisticRegression(penalty = 'l2', C = 1.0) #enforcing data regularisation with the penalty argument set to L2
logreg.fit(x_train, y_train)
logreg_predict = logreg.predict(x_test)
logreg_acc = accuracy_score(logreg_predict, y_test)
print("The model's accuracy is {: .2f}".format(logreg_acc*100))

The model's accuracy is  87.78


In [ ]:
print(confusion_matrix(y_test, logreg_predict))
print("\n")
print(classification_report(y_test, logreg_predict))

[[2139  198]
 [ 344 1756]]


              precision    recall  f1-score   support

         0.0       0.86      0.92      0.89      2337
         1.0       0.90      0.84      0.87      2100

    accuracy                           0.88      4437
   macro avg       0.88      0.88      0.88      4437
weighted avg       0.88      0.88      0.88      4437



<h2>Hyperparameter Tuning</h2>

In [25]:
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [26]:
param_grid = {'C':[100, 10, 1.0, 0.1, 0.01], 'solver':['newton-cg', 'lbfgs', 'liblinear']}
grid = GridSearchCV(LogisticRegression(), param_grid, cv = 5)
grid.fit(x_train, y_train)
print("Best cross-validation score: {:.2f}\n".format(grid.best_score_))
print("Best parameters: {}".format(grid.best_params_))

Best cross-validation score: 0.92

Best parameters: {'C': 100, 'solver': 'liblinear'}


In [27]:
y_pred = grid.predict(x_test)

In [28]:
logreg_acc2 = accuracy_score(y_pred, y_test)
print("Test accuracy: {:.2f}%".format(logreg_acc2*100))

Test accuracy: 93.16%


In [29]:
savedacc = "{:.2f}%".format(logreg_acc2*100)
savedacc

'93.16%'

In [30]:
pickle.dump(savedacc, open("hatemodel/services/accuracy.pkl", 'wb'))

In [31]:
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

[[2207  149]
 [ 155 1934]]


              precision    recall  f1-score   support

         0.0       0.93      0.94      0.94      2356
         1.0       0.93      0.93      0.93      2089

    accuracy                           0.93      4445
   macro avg       0.93      0.93      0.93      4445
weighted avg       0.93      0.93      0.93      4445



<h2>Testing the model</h2>

In [35]:
input_data = lemmatizing(clean("He hates you."))
input_data

'hates'

In [36]:
input_text = vect2.transform(input_data.split(" "))
input_text

<1x239648 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [37]:
grid.predict(input_text)

array([0.])

<h2> Exporting trained model into pickle file</h2>

In [38]:
# Export the Trained Model using Pickle
pickle.dump(grid, open('hatemodel/services/savedmodel.pkl', 'wb'))